In [1]:
import numpy as np
import pandas as pd
import json
from typing import Dict
from itertools import groupby
from functools import reduce
from collections import Counter, defaultdict
import re

In [2]:
df = pd.read_json("export/export_0.json")

In [3]:
# max_stage = 7

# Gold Per Stage

In [4]:
def gold_spent(player):
    gold_spent_per_round = {}
    for currRound, group in groupby(player["gold"]["by_round"], lambda x: x["current_round"]):
        l = list(group)
        goldSpent = max(0,int(l[0]["gold"]) - int(l[-1]["gold"]))
        gold_spent_per_round[currRound] = goldSpent
    gold_spent_by_stage = defaultdict(int)
    for x in list(gold_spent_per_round.keys()):
        m = re.match(r'(\d).+', x)
        gold_spent_by_stage[int(m.group(1))] += gold_spent_per_round[m.group(0)]

    return gold_spent_by_stage

In [5]:
gold_spent(df.player.iloc[0])

defaultdict(int, {1: 4, 2: 20, 3: 13, 4: 97, 5: 40, 6: 30})

In [6]:
gold_dict = [gold_spent(df.player.iloc[i]) for i in range(df.shape[0])]

In [7]:
gold_dict[:10]

[defaultdict(int, {1: 4, 2: 20, 3: 13, 4: 97, 5: 40, 6: 30}),
 defaultdict(int, {1: 4, 2: 5, 3: 27, 4: 67, 5: 68, 6: 87}),
 defaultdict(int, {1: 2, 2: 5, 3: 86, 4: 24, 5: 70, 6: 12}),
 defaultdict(int, {1: 4, 2: 7, 3: 52, 4: 68, 5: 109, 6: 31}),
 defaultdict(int, {1: 2, 2: 11, 3: 36, 4: 93, 5: 66}),
 defaultdict(int, {1: 2, 2: 22, 3: 15, 4: 62, 5: 80}),
 defaultdict(int, {1: 4, 2: 4, 3: 53, 4: 71, 5: 76}),
 defaultdict(int, {1: 4, 2: 13, 3: 24, 4: 69, 5: 87}),
 defaultdict(int, {1: 2, 2: 9, 3: 52, 4: 79, 5: 94, 6: 18}),
 defaultdict(int, {1: 1, 2: 25, 3: 34, 4: 47, 5: 43, 6: 0})]

In [8]:
gold_df = pd.DataFrame(gold_dict).fillna(0)
gold_df.head()

,1,2,3,4,5,6,7
0,4.0,20.0,13.0,97.0,40.0,30.0,0.0
1,4.0,5.0,27.0,67.0,68.0,87.0,0.0
2,2.0,5.0,86.0,24.0,70.0,12.0,0.0
3,4.0,7.0,52.0,68.0,109.0,31.0,0.0
4,2.0,11.0,36.0,93.0,66.0,0.0,0.0


# Level Per Stage

In [9]:
def level_stage(player):
    tp_dict = {}
    
    level_agg = 0
    current_stage = 0
    count_stage = 0
    
    for key, group in groupby(player["xp"]["by_round"], lambda x: x["current_round"]): 
        round_split = key.split("-")
        stage = int(round_split[0])

        #start of loop
        if current_stage == 0:
            current_stage = stage
        #going from stage x to stage y
        elif current_stage != stage:
            level_avg = level_agg/count_stage
#             tp_dict[f"level_at_stage_{current_stage}"] = level_avg 
            tp_dict[current_stage] = level_avg 
            current_stage = stage
            level_agg = 0
            count_stage = 0

        #continue
        list_r = list(group)
        level_agg += list_r[0]['level']
        count_stage += 1

    #getting the last stage
    level_avg = level_agg/count_stage
#     tp_dict[f"level_at_stage_{current_stage}"] = level_avg
    tp_dict[current_stage] = level_avg
    
    return tp_dict

In [10]:
level_stage(df.player.iloc[0])

{1: 2.0,
 2: 4.166666666666667,
 3: 5.333333333333333,
 4: 6.5,
 5: 7.333333333333333,
 6: 8.0}

In [11]:
level_dict = [level_stage(df.player.iloc[i]) for i in range(df.shape[0])]

In [12]:
level_df = pd.DataFrame(level_dict).fillna(method='ffill',axis=1)
level_df.head()

,1,2,3,4,5,6,7
0,2.000000,4.166667,5.333333,6.500000,7.333333,8.00,8.00
1,2.000000,4.333333,5.000000,5.833333,7.166667,8.00,8.00
2,1.666667,3.666667,5.166667,6.000000,6.833333,7.00,7.00
3,1.666667,4.166667,5.833333,7.333333,8.000000,8.00,8.00
4,1.666667,3.500000,4.833333,6.000000,7.750000,7.75,7.75


# Active Traits

In [13]:
def encode_traits(trait_dict_column, active_trait_df, trait_index):
    encoding_list = []
    for trait_dict in trait_dict_column:
        trait_encoding = [ 0 for i in range(len(trait_index)) ]
        for key, value in trait_dict.items():
            if key[:5] == 'Set3_':
                trait = key[5:]
            else:
                trait = key
            min_active_dict = active_trait_df.loc[active_trait_df['trait'] == trait]['min_active'].values[0]
            has_trait_flag = 0
            for k, v in min_active_dict.items():
                if value >= v[0] and value < v[1]:
                    trait_tier = int(k)
                    has_trait_flag = 1
            if has_trait_flag == 0:
                trait_tier = 0
            trait_encoding[trait_index[trait]] = trait_tier
        encoding_list.append(trait_encoding)
    return encoding_list

def get_active_traits(player, char_trait, trait_tier):
    active_traits = {}
    last_rounds = get_last_rounds(player)
    for rounds in last_rounds:
        stage = rounds.split('-')[0]
        board = player['round_outcomes']['by_round'][rounds]['board']
        char_seen = set()
        trait_count = {}
        for char_dict in board:
            char = char_dict['character_id']
            if char not in char_seen:
                char_seen.add(char_dict['character_id'])
                trait_list = char_trait.loc[char_trait['name'] == char]['trait'].values[0].replace('\'', '').replace(' ', '').strip('][ ').split(',')
                for trait in trait_list:
                    if trait not in trait_count:
                        trait_count[trait] = 1
                    else:
                        trait_count[trait] += 1      
        active_traits[stage] = trait_count
        
    return active_traits
    
def get_last_rounds(player):
    round_list = list(player['round_outcomes']['by_round'].keys())
    last_round_stage = []
    last_r = 0
    last_round = ''
    for rounds in round_list:
        r = int(rounds.split('-')[1])
        if last_r > r:
            last_round_stage.append(last_round)
        last_r = r
        last_round = rounds
    last_round_stage.append(round_list[-1])
    return last_round_stage

In [14]:
with open('full_mapping.json') as json_data:
    fullmap = json.load(json_data)
    
char_trait = pd.DataFrame(fullmap['character_trait_json'].items()).astype(str)
char_trait.columns = ['name', 'trait']
print(char_trait)

trait_tier = pd.DataFrame(fullmap['trait_tier_mapping_json'].items())
print(trait_tier)

                name                                          trait
0        TFT3_Thresh                       ['ManaReaver', 'Chrono']
1          TFT3_Ekko                  ['Infiltrator', 'Cybernetic']
2   TFT3_AurelionSol                          ['Starship', 'Rebel']
3          TFT3_Lulu                        ['Mystic', 'Celestial']
4         TFT3_Janna                    ['Paragon', 'StarGuardian']
5        TFT3_Soraka                     ['Mystic', 'StarGuardian']
6         TFT3_Urgot                    ['Protector', 'Battlecast']
7     TFT3_Gangplank  ['Mercenary', 'Demolitionist', 'SpacePirate']
8        TFT3_Xerath                       ['Sorcerer', 'DarkStar']
9          TFT3_Fizz                   ['Infiltrator', 'MechPilot']
10       TFT3_Irelia    ['ManaReaver', 'Blademaster', 'Cybernetic']
11       TFT3_Ezreal                          ['Blaster', 'Chrono']
12        TFT3_Riven                      ['Blademaster', 'Chrono']
13        TFT3_Fiora                  ['Blademas

In [15]:
get_active_traits(df.player.iloc[0], char_trait, trait_tier)

{'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1},
 '2': {'Blademaster': 1,
  'Celestial': 2,
  'Mystic': 1,
  'DarkStar': 3,
  'Protector': 2,
  'Vanguard': 1},
 '3': {'Blademaster': 1,
  'Celestial': 2,
  'Sniper': 1,
  'Chrono': 2,
  'Vanguard': 2,
  'Protector': 2,
  'DarkStar': 2},
 '4': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 2,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'DarkStar': 1,
  'StarGuardian': 1},
 '5': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 3,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'Battlecast': 1,
  'DarkStar': 1,
  'Chrono': 1},
 '6': {'Sniper': 1,
  'Celestial': 4,
  'Blademaster': 3,
  'ManaReaver': 1,
  'Cybernetic': 1,
  'Protector': 4,
  'Battlecast': 1,
  'DarkStar': 1,
  'Chrono': 1}}

In [16]:
def active_trait_func(player):
    return get_active_traits(player, char_trait, trait_tier)

In [17]:
active_trait_dict = [active_trait_func(df.player.iloc[i]) for i in range(df.shape[0])]

In [18]:
active_trait_dict[:2]

[{'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1},
  '2': {'Blademaster': 1,
   'Celestial': 2,
   'Mystic': 1,
   'DarkStar': 3,
   'Protector': 2,
   'Vanguard': 1},
  '3': {'Blademaster': 1,
   'Celestial': 2,
   'Sniper': 1,
   'Chrono': 2,
   'Vanguard': 2,
   'Protector': 2,
   'DarkStar': 2},
  '4': {'Sniper': 1,
   'Celestial': 4,
   'Blademaster': 2,
   'ManaReaver': 1,
   'Cybernetic': 1,
   'Protector': 4,
   'DarkStar': 1,
   'StarGuardian': 1},
  '5': {'Sniper': 1,
   'Celestial': 4,
   'Blademaster': 3,
   'ManaReaver': 1,
   'Cybernetic': 1,
   'Protector': 4,
   'Battlecast': 1,
   'DarkStar': 1,
   'Chrono': 1},
  '6': {'Sniper': 1,
   'Celestial': 4,
   'Blademaster': 3,
   'ManaReaver': 1,
   'Cybernetic': 1,
   'Protector': 4,
   'Battlecast': 1,
   'DarkStar': 1,
   'Chrono': 1}},
 {'1': {'Vanguard': 2, 'Cybernetic': 1, 'StarGuardian': 1},
  '2': {'Blaster': 2,
   'Chrono': 1,
   'Mystic': 1,
   'Battlecast': 4,
   'Infiltrator': 1,
   'Braw

In [19]:
active_trait_df = pd.DataFrame(active_trait_dict).fillna(method='ffill',axis=1)
active_trait_df.head()

,1,2,3,4,5,6,7
0,"{'Blademaster': 1, 'Cybernetic': 1, 'Protector...","{'Blademaster': 1, 'Celestial': 2, 'Mystic': 1...","{'Blademaster': 1, 'Celestial': 2, 'Sniper': 1...","{'Sniper': 1, 'Celestial': 4, 'Blademaster': 2...","{'Sniper': 1, 'Celestial': 4, 'Blademaster': 3...","{'Sniper': 1, 'Celestial': 4, 'Blademaster': 3...","{'Sniper': 1, 'Celestial': 4, 'Blademaster': 3..."
1,"{'Vanguard': 2, 'Cybernetic': 1, 'StarGuardian...","{'Blaster': 2, 'Chrono': 1, 'Mystic': 1, 'Batt...","{'Blaster': 2, 'Battlecast': 4, 'Infiltrator':...","{'Sorcerer': 1, 'Battlecast': 4, 'Blaster': 2,...","{'Infiltrator': 1, 'Battlecast': 5, 'Sorcerer'...","{'Infiltrator': 1, 'Battlecast': 6, 'Mystic': ...","{'Infiltrator': 1, 'Battlecast': 6, 'Mystic': ..."
2,"{'Sniper': 1, 'Chrono': 1, 'Protector': 1, 'Da...","{'Blademaster': 1, 'Celestial': 2, 'Mystic': 1...","{'Protector': 2, 'Celestial': 2, 'Blademaster'...","{'Protector': 2, 'Celestial': 2, 'Blademaster'...","{'Protector': 2, 'Celestial': 2, 'Blademaster'...","{'Protector': 2, 'Celestial': 2, 'Blademaster'...","{'Protector': 2, 'Celestial': 2, 'Blademaster'..."
3,"{'Mystic': 1, 'Battlecast': 2, 'Infiltrator': ...","{'Infiltrator': 1, 'DarkStar': 1, 'Mystic': 1,...","{'Mystic': 2, 'Astro': 1, 'Battlecast': 4, 'So...","{'Sniper': 4, 'Celestial': 1, 'Chrono': 1, 'As...","{'Sniper': 4, 'Astro': 3, 'DarkStar': 2, 'Myst...","{'Sniper': 4, 'Astro': 3, 'DarkStar': 1, 'Star...","{'Sniper': 4, 'Astro': 3, 'DarkStar': 1, 'Star..."
4,"{'Vanguard': 2, 'StarGuardian': 1, 'Cybernetic...","{'Sniper': 1, 'Celestial': 1, 'Vanguard': 2, '...","{'Sniper': 2, 'Chrono': 1, 'Infiltrator': 1, '...","{'Sniper': 2, 'Chrono': 2, 'Infiltrator': 1, '...","{'Mystic': 1, 'DarkStar': 6, 'Sniper': 3, 'Inf...","{'Mystic': 1, 'DarkStar': 6, 'Sniper': 3, 'Inf...","{'Mystic': 1, 'DarkStar': 6, 'Sniper': 3, 'Inf..."


# Characters and Items

In [20]:
def character_items(player):
    tp_dict={}
    
    round_outcome_dict = player['round_outcomes']
    by_round_dict = round_outcome_dict['by_round']
    
    current_stage = 0
    prev_stage = 1
    
    current_round = 'a'
    prev_round = 'a'
    
    for key in by_round_dict.keys():
        round_split = key.split("-")
        current_stage = int(round_split[0])

        current_round = key

        if(prev_round == 'a'):
            prev_round = key
            prev_stage = current_stage

        if (current_stage == prev_stage):
            prev_round = key
        #went from stage x to stage y
        elif (current_stage != prev_stage):
            #a dictionary containg board
            #get the board characters and items
            board_dict = by_round_dict[prev_round]['board']
            #each index has one character and item combo in a dictionary with character_id and items
            #the temp dictionary will be 1: [character_id, items]

            temp_list = []
            
            for char in board_dict:
                character_id_value = char['character_id']
                items_value = char['items']

                temp_dict = {}
                temp_dict['character_id'] = character_id_value
                temp_dict['items'] = items_value

                temp_list.append(temp_dict)

            tp_dict[prev_stage] = temp_list

            prev_round = key
            prev_stage = current_stage
            
    #last one
    board_dict = by_round_dict[prev_round]['board']
    
    temp_list = []

    for char in board_dict:
        character_id_value = char['character_id']
        items_value = char['items']

        temp_dict = {}
        temp_dict['character_id'] = character_id_value
        temp_dict['items'] = items_value

        temp_list.append(temp_dict)

    tp_dict[prev_stage] = temp_list
    
    
    return tp_dict


In [21]:
char_item_dict = [character_items(df.player.iloc[i]) for i in range(df.shape[0])]

In [22]:
char_item_dict[2]

{1: [{'character_id': 'TFT3_Caitlyn', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': []}],
 2: [{'character_id': 'TFT3_Xayah', 'items': [6, 19]},
  {'character_id': 'TFT3_Karma', 'items': []},
  {'character_id': 'TFT3_XinZhao', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [5]}],
 3: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_Xayah', 'items': [6, 19]},
  {'character_id': 'TFT3_Riven', 'items': []},
  {'character_id': 'TFT3_Shen', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [5]},
  {'character_id': 'TFT3_Fiora', 'items': []}],
 4: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_Riven', 'items': []},
  {'character_id': 'TFT3_Xayah', 'items': [12, 19, 69]},
  {'character_id': 'TFT3_Shen', 'items': []},
  {'character_id': 'TFT3_JarvanIV', 'items': [55]},
  {'character_id': 'TFT3_MasterYi', 'items': []}],
 5: [{'character_id': 'TFT3_Rakan', 'items': [37]},
  {'character_id': 'TFT3_MasterYi', 'item

In [23]:
# res = []
# for row in char_item_dict:
#     for stage,char_items in row.items():
#         for x in char_items:
#             res.append((stage, x["character_id"], x["items"]))

In [24]:
#  char_item_stage = pd.DataFrame(res,columns=["stage","character_id","items"])

In [25]:
# char_item_stage.head()

In [26]:
# l = []
# for i,stage in enumerate(char_item_stage.stage.iloc[:5]):
#     gold_spent = {f"gold_spent_stage_{j}" : gold_dict[i][j] for j in range(1,stage+1)}
#     player_level = {f"player_level_stage_{j}" : level_dict[i][j]  for j in range(1,stage+1)}
#     active_trait = {f"active_trait_stage_{j}" : active_trait_dict[i][str(j)]  for j in range(1,stage+1)}
#     l.append({"stage" : stage, "character_id" : char_item_stage['character_id'].iloc[4], })
#     print(f"i : {i}, stage : {stage}, character : {char_item_stage['character_id'].iloc[4]}")
# #     print(*[f"gold_spent_stage_{j} : {gold_dict[i][j]}" for j in range(1,stage+1)])
# #     print(*[f"player_level_stage_{j} : {level_dict[i][j]}"  for j in range(1,stage+1)])
# #     print(*[f"active_trait_stage_{j} : {active_trait_dict[i][str(j)]}"  for j in range(1,stage+1)])
#     print(gold_spent, player_level, active_trait)
        

In [27]:
level_dict[0]

{1: 2.0,
 2: 4.166666666666667,
 3: 5.333333333333333,
 4: 6.5,
 5: 7.333333333333333,
 6: 8.0}

In [28]:
i = 0
res = []
for i in range(len(char_item_dict)):
    for stage,comp in char_item_dict[i].items():
        for char in comp:
            res.append({"stage" : stage, "character_id" : char['character_id'], "items" : char["items"], \
                "gold_spent" : gold_df.iloc[i][stage], "level" : level_df.iloc[i][stage], \
                "active_trait" : active_trait_df.iloc[i][str(stage)]})
#             print(f"i : {i}")
#             print(f"stage : {stage}")
#             print(f"character_id : {char['character_id']}")
#             print(char["items"])
#             print(gold_df.iloc[i][stage])      
#             print(level_df.iloc[i][stage])
#             print(active_trait_df.iloc[i][str(stage)])
    



In [29]:
combined_df = pd.DataFrame(res)
combined_df.head()

,stage,character_id,items,gold_spent,level,active_trait
0,1,TFT3_Fiora,[],4.0,2.000000,"{'Blademaster': 1, 'Cybernetic': 1, 'Protector..."
1,1,TFT3_JarvanIV,[],4.0,2.000000,"{'Blademaster': 1, 'Cybernetic': 1, 'Protector..."
2,2,TFT3_Xayah,[19],20.0,4.166667,"{'Blademaster': 1, 'Celestial': 2, 'Mystic': 1..."
3,2,TFT3_Karma,[],20.0,4.166667,"{'Blademaster': 1, 'Celestial': 2, 'Mystic': 1..."
4,2,TFT3_XinZhao,[],20.0,4.166667,"{'Blademaster': 1, 'Celestial': 2, 'Mystic': 1..."
